In [ ]:
import openturns as ot
from math import sqrt

# 1. The function G
def functionCrue(X) :
    Hd = 3.0
    Zb = 55.5
    L = 5.0e3
    B = 300.0
    Zd = Zb + Hd
    Q, Ks, Zv, Zm = X
    alpha = (Zm - Zv)/L
    H = (Q/(Ks*B*sqrt(alpha)))**(3.0/5.0)
    Zc = H + Zv
    S = Zc - Zd
    return [S]

# Creation of the problem function
g = ot.PythonFunction(4, 1, functionCrue)
g = ot.MemoizeFunction(g)

# 2. Random vector definition
myParam = ot.GumbelAB(1013., 558.)
Q = ot.ParametrizedDistribution(myParam)
otLOW = ot.TruncatedDistribution.LOWER
Q = ot.TruncatedDistribution(Q, 0, otLOW)
Ks = ot.Normal(30.0, 7.5)
Ks = ot.TruncatedDistribution(Ks, 0, otLOW)
Zv = ot.Uniform(49.0, 51.0)
Zm = ot.Uniform(54.0, 56.0)

# 3. Create the joint distribution function
distribution = ot.ComposedDistribution([Q, Ks, Zv, Zm])

# 4. Cree le plan d'expériences (SobolIndicesExperiment existe dans OT 1.13)
size = 50
experiment = ot.SobolIndicesExperiment(distribution, size)
inputDesign = experiment.generate()
outputDesign = g(inputDesign)
muY = outputDesign.computeMean()

# 5. Estime les indices
# 5.1 Classiques
myClassicalSobolStudy = ot.MauntzKucherenkoSensitivityAlgorithm(inputDesign, outputDesign, size)
# 5.2 Iteratifs
myIterativeSobolStudy = ot.MauntzKucherenkoSobolIndices(g.getInputDimension(), g.getOutputDimension())
# increment will do 50 iterations from the sample
myIterativeSobolStudy.incrementIndices(outputDesign - muY)

# 6. Recupère les indices du premier ordre
print(myClassicalSobolStudy.getFirstOrderIndices())
print(myIterativeSobolStudy.getFirstOrderIndices())

# 7. Recupère les indices d'ordre total
print(myClassicalSobolStudy.getTotalOrderIndices())
print(myIterativeSobolStudy.getTotalOrderIndices())